# RedrugAI

### Intro

### Set up
#### Biquery 
1. Create Biquery project and dataset
2. Create remote vertex model (for text embeddings)
   https://cloud.google.com/bigquery/docs/generate-text-embedding#console_1
#### Python
1. version >= 3.12.7

In [15]:
%pip install --upgrade bigframes --progress-bar pretty

Note: you may need to restart the kernel to use updated packages.



Usage:   
  c:\Users\b0220\Documents\ReDrugAI\.venv\Scripts\python.exe -m pip install [options] <requirement specifier> [package-index-options] ...
  c:\Users\b0220\Documents\ReDrugAI\.venv\Scripts\python.exe -m pip install [options] -r <requirements file> [package-index-options] ...
  c:\Users\b0220\Documents\ReDrugAI\.venv\Scripts\python.exe -m pip install [options] [-e] <vcs project url> ...
  c:\Users\b0220\Documents\ReDrugAI\.venv\Scripts\python.exe -m pip install [options] [-e] <local project path> ...
  c:\Users\b0220\Documents\ReDrugAI\.venv\Scripts\python.exe -m pip install [options] <archive url/path> ...

option --progress-bar: invalid choice: 'pretty' (choose from 'auto', 'on', 'off', 'raw')


In [1]:
import bigframes.pandas as bpd
region = "US"
project_id = "redrugai"
dataset_id = "redrugai_data"
embedding_model_name = "biolord"
source_project_id = "bigquery-public-data"
source_dataset_id = "open_targets_platform"
# Configure BigQuery client  
bpd.options.bigquery.project = project_id
bpd.options.bigquery.location = region


## Prepare for the disease embedding table
- only need to do once
- the purpose is to pre-built the embedding slim table for doing vector search at public dataset

In [2]:
# Execute SQL to create disease embeddings table
source_table_name = "disease"
embedding_table_name = f"{source_table_name}_embedding"

# Fixed query with proper content column
query_create_embeddings = f"""
CREATE OR REPLACE TABLE `{project_id}.{dataset_id}.{embedding_table_name}` AS
WITH source_table AS (
  SELECT
    id,
    name,
    synonyms,
    CONCAT(
      COALESCE(name, ''), ' ',
      COALESCE(description, '')
    ) AS content
  FROM
    `{source_project_id}.{source_dataset_id}.{source_table_name}`
  LIMIT 50
)
SELECT
  s.id,
  s.name,
  s.synonyms,
  e.ml_generate_embedding_result AS embedding
FROM
  source_table s
JOIN
  ML.GENERATE_EMBEDDING(
    MODEL `{project_id}.{dataset_id}.{embedding_model_name}`,
    (SELECT id, content FROM source_table),
    STRUCT(TRUE AS flatten_json_output)
  ) e
ON s.id = e.id
"""

# Execute the query to create the embeddings table
bpd.read_gbq(query_create_embeddings)
print("Disease embeddings table created successfully!")


Disease embeddings table created successfully!


### Build index to optimize search vector for rows > 1M (skip temporarily)

In [ ]:
import bigframes.bigquery as bbq

# Create a vector index for efficient searching on the disease embeddings table
full_table_id = f"{project_id}.{dataset_id}.{embedding_table_name}"

bbq.create_vector_index(
    table_id=full_table_id,
    column_name='embedding',
)


## Vector Search

### First layer

In [21]:
base_table_name = "disease_embedding"
text_embedding_model_name = "embedding005"
query_text = "chronic kidney disease"

vector_search_query = f"""
WITH query_table AS (
    SELECT *
    FROM ML.GENERATE_EMBEDDING(
        MODEL `{project_id}.{dataset_id}.{text_embedding_model_name}`,
        (SELECT '{query_text}' AS content)
    )
)
SELECT
    base.id,
    base.name AS disease_name,
    query.content AS query_text,
    distance
FROM
    VECTOR_SEARCH(
        TABLE `{project_id}.{dataset_id}.{base_table_name}`,
        'embedding',
        (SELECT * FROM query_table),
        'ml_generate_embedding_result',
        top_k => 3,
        distance_type => 'COSINE'
    )
"""

# Execute the search query
search_results_df = bpd.read_gbq(vector_search_query)
search_results_df = search_results_df.sort_values('distance', ascending=False)

# Display the results
print("Vector Search Results:")
print(search_results_df.head())

Vector Search Results:


           id           disease_name              query_text  distance
2  GO_0003094  glomerular filtration  chronic kidney disease  0.437125
1  GO_0003092  renal water retention  chronic kidney disease  0.390453
0  GO_0003014   renal system process  chronic kidney disease  0.372117

[3 rows x 4 columns]


### Second Layer

In [9]:
# get the drugs from the disease ids
# Extract the disease IDs from search results
disease_ids = search_results_df['id'].tolist()
disease_ids_str = "', '".join(disease_ids)

# Query to get drugs associated with these diseases
drug_query = f"""
SELECT 
    kd.diseaseId as disease_id,
    kd.drugId as drug_id,
    dm.name as drug_name
FROM 
    `{source_project_id}.{source_dataset_id}.known_drug` kd
JOIN 
    `{source_project_id}.{source_dataset_id}.drug_molecule` dm
ON 
    kd.drugId = dm.id
WHERE 
    kd.diseaseId IN ('GO_0000050')
ORDER BY 
    kd.diseaseId, dm.name
"""

# Execute the query
drug_results_df = bpd.read_gbq(drug_query)

print("Drugs associated with similar diseases:")
print(drug_results_df.head(10))

Drugs associated with similar diseases:


Empty DataFrame
Columns: [disease_id, drug_id, drug_name]
Index: []

[0 rows x 3 columns]


### Thrid Layer

In [47]:
# Filter out drugs whose side effects conflict with the researched disease

research_disease = 'MONDO_0005147'
drug_id_list = ['CHEMBL4297879', 'CHEMBL4297879']
text_generation_model_name = 'gemini25pro'
adverse_drug_table = 'openfda_significant_adverse_drug_reactions'

# Prepare drug IDs for query
drug_ids_str = "', '".join(drug_id_list)

# SINGLE INTEGRATED QUERY - Everything in one SQL statement
query = f"""
-- 1. Fetch adverse events for specified drug IDs from openfda table
WITH adverse_events AS (
  SELECT 
    chembl_id as drug_id,
    event as drug_adverse_event,
  FROM 
    `{source_project_id}.{source_dataset_id}.{adverse_drug_table}`
  WHERE 
    chembl_id IN ('{drug_ids_str}')
),

-- 2. Get description from research disease
research_disease_info AS (
  SELECT 
    id as disease_id,
    name as disease_name,
    description as disease_description
  FROM 
    `{source_project_id}.{source_dataset_id}.disease`
  WHERE 
    id = '{research_disease}'
),

-- 3. Combine adverse events with disease info for ML analysis
analysis_data AS (
  SELECT 
    ae.drug_id,
    ae.drug_adverse_event,
    rd.disease_id,
    rd.disease_name,
    rd.disease_description,
    CONCAT(
      'Analyze the relationship between the adverse drug event "', ae.drug_adverse_event, 
      '" and the disease "', rd.disease_name, 
      '" (', COALESCE(rd.disease_description, 'No description available'), 
      '). Determine if this adverse event could be: 1) Beneficial for treating the disease, ', 
      '2) Harmful/contraindicated, or 3) Neutral/unrelated. Provide a concise 2-3 sentence explanation.'
    ) as prompt
  FROM 
    adverse_events ae
  CROSS JOIN 
    research_disease_info rd
)

-- 4. Final result with ML.GENERATE_TEXT inference as new column
SELECT
  drug_id,
  drug_adverse_event,
  disease_id, 
  disease_name,
  disease_description,
  prompt,
  ml_generate_text_llm_result as infernece
FROM
  ML.GENERATE_TEXT(
    MODEL redrugai.redrugai_data.gemini25pro,
    (SELECT * FROM analysis_data),
    STRUCT(
      0.2 AS temperature,
      1024 AS max_output_tokens,
      TRUE AS flatten_json_output
    )
  ) 
"""

results_df = bpd.read_gbq(query)
print(results_df)

         drug_id                 drug_adverse_event     disease_id  \
0  CHEMBL4297879                     lung infection  MONDO_0005147   
1  CHEMBL4297879                             chills  MONDO_0005147   
2  CHEMBL4297879                         chest pain  MONDO_0005147   
3  CHEMBL4297879                    nasopharyngitis  MONDO_0005147   
4  CHEMBL4297879                   chest discomfort  MONDO_0005147   
5  CHEMBL4297879                          pneumonia  MONDO_0005147   
6  CHEMBL4297879                              cough  MONDO_0005147   
7  CHEMBL4297879              product closure issue  MONDO_0005147   
8  CHEMBL4297879  upper respiratory tract infection  MONDO_0005147   
9  CHEMBL4297879          infusion related reaction  MONDO_0005147   

               disease_name  \
0  type 1 diabetes mellitus   
1  type 1 diabetes mellitus   
2  type 1 diabetes mellitus   
3  type 1 diabetes mellitus   
4  type 1 diabetes mellitus   
5  type 1 diabetes mellitus   
6  type 1 di